# 第8章：ニューラルネット

> 第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

In [ ]:
import os
import collections
import shutil
import string
import random
from pprint import pprint
import pickle
import time

from gensim.models import KeyedVectors
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
DATADIR = "data"          # データを保存するおおもとのディレクトリ
CURRENTDIR = "/workspace/notebook" # notebookディレクトリへのパス
CHPDIR = os.path.join(DATADIR, "chapter8")
CHP6DIR = os.path.join(DATADIR, "chapter6")
CHP7DIR = os.path.join(DATADIR, "chapter7")

try:
    os.mkdir(CHPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

## 70. 単語ベクトルの和による特徴量

問題50で構築したデータをベクトルに変換する。\
文章の単語をすべて単語ベクトルに置き換えたのち、平均をとる形で実装する。

In [ ]:
# 利用するデータをchapter6のディレクトリから読み込む
fnames = ["train.txt", "valid.txt", "test.txt"]
chp6dir = os.path.join(DATADIR, "chapter6")

all_data = {}

for fname in fnames:
    temp_fpath = os.path.join(CHP6DIR, fname)
    with open(temp_fpath, "r", encoding="utf8")as fr:
        # タブ区切りをリストで取得
        data = [line.rstrip("\n").split("\t") for line in fr]
        all_data[fname] = data

# 各カテゴリの事例数を確認する
for fname, data in all_data.items():
    print("【{}】".format(fname))
    pprint(collections.Counter([record[0] for record in data]))

# ラベルベクトルの作成
categorys = {"b": 0, "t": 1, "e":2, "m":3}

### ------------↓以下の作業は初回実行時のみ行う↓------------

In [ ]:
# word2vecモデルを読み込む
# モデルの読み込み（割と時間かかる）
if input("実行しますか？（y/n）2回目以降はpickleを読み込むセルから実行"):
    model_path = os.path.join(CHP7DIR, "GoogleNews-vectors-negative300.bin.gz")
    model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
# 文書→ベクトル関数の作成
def transform_d2v(text):
    table = str.maketrans(string.punctuation, " "*len(string.punctuation))
    words = text.translate(table).split()  # 記号をスペースに置換後、スペースで分割してリスト化
    vec = [model[word] for word in words if word in model]  # 1語ずつベクトル化
    
    return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [ ]:
# 特徴ベクトル化実行
X_train = torch.stack([transform_d2v(doc_data[1]) for doc_data in all_data["train.txt"]])
X_valid = torch.stack([transform_d2v(doc_data[1]) for doc_data in all_data["valid.txt"]])
X_test = torch.stack([transform_d2v(doc_data[1]) for doc_data in all_data["test.txt"]])

print(X_train.shape, X_valid.shape, X_test.shape)

In [ ]:
y_train = torch.tensor([categorys[doc_data[0]] for doc_data in all_data["train.txt"]])
y_valid = torch.tensor([categorys[doc_data[0]] for doc_data in all_data["valid.txt"]])
y_test = torch.tensor([categorys[doc_data[0]] for doc_data in all_data["test.txt"]])

print(y_train.shape)
print(y_train)

In [ ]:
# 保存
pt_files = ["X_train.pt", "X_valid.pt", "X_test.pt", "y_train.pt", "y_valid.pt", "y_test.pt"]
save_objs = [X_train, X_valid, X_test, y_train, y_valid, y_test]

for pt_file, save_obj in zip(pt_files, save_objs):
    output_path = os.path.join(CHPDIR, pt_file)
    torch.save(save_obj, output_path)

### ------------↑ここまで↑------------

### ------------↓2回目以降の作業↓------------


In [ ]:
# 各ベクトルのロード
X_train = torch.load(os.path.join(CHPDIR, "X_train.pt"))
X_valid = torch.load(os.path.join(CHPDIR, "X_valid.pt"))
X_test = torch.load(os.path.join(CHPDIR, "X_test.pt"))
y_train = torch.load(os.path.join(CHPDIR, "y_train.pt"))
y_valid = torch.load(os.path.join(CHPDIR, "y_valid.pt"))
y_test = torch.load(os.path.join(CHPDIR, "y_test.pt"))

# 確認
print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

## 71. 単層ニューラルネットワークによる予測

70で作成したベクトルについてソフトマックスによる計算を行う

In [ ]:
# レイヤーの定義
class SLPNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(SLPNet, self).__init__()
        self.fc = nn.Linear(input_size, output_size, bias=False)
        nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化
    
    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
# 1行分投入
model = SLPNet(300, 4)  # 単層ニューラルネットワークの初期化
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
print(y_hat_1)

In [ ]:
# 4行分投入
Y_hat = torch.softmax(model.forward(X_train[:4]), dim=-1)
print(Y_hat)

## 72. 損失と勾配の計算

クロスエントロピー損失と行列Wに対する勾配を求める

In [ ]:
# 交差エントロピー誤差の用意
criterion = nn.CrossEntropyLoss()

# ソフトマックスだけ行ったものをラベルと比較
l_1 = criterion(model(X_train[:1]), y_train[:1])
# 勾配の初期化
model.zero_grad()
# 勾配の計算
l_1.backward()
print(f"損失:{l_1:.4f}")
print(f"勾配:{model.fc.weight.grad}")

In [ ]:
# ソフトマックスだけ行ったものをラベルと比較
l_1 = criterion(model(X_train[:4]), y_train[:4])
# 勾配の初期化
model.zero_grad()
# 勾配の計算
l_1.backward()
print(f"損失:{l_1:.4f}")
print(f"勾配:{model.fc.weight.grad}")

## 73. 確率的勾配降下法による学習

> 確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列WWを学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [ ]:
# Datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# DataLoaderの作成
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

dataloader_train

In [ ]:
# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義（確率的勾配降下法）
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# 学習
num_epochs = 10
for epoch in range(num_epochs):
    # 訓練モードにする
    model.train()
    loss_train = 0.0
    for i, (inputs, labels) in enumerate(dataloader_train):
        # 勾配を0で初期化
        optimizer.zero_grad()
        
        # 順伝播 + 誤差逆伝播 + 重み更新
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # 損失を記憶
        loss_train += loss.item()
        
    # バッチ単位の平均損失計算
    loss_train = loss_train / i
    
    # 検証データの損失計算
    model.eval()
    with torch.no_grad():
        inputs, labels = next(iter(dataloader_valid))
        outputs = model(inputs)
        loss_valid = criterion(outputs, labels)
        
    print(f"epoch: {epoch + 1}, loss_train: {loss_train:.4f}, loss_valid: {loss_valid:.4f}")

## 74. 正解率の計測

> 問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [ ]:
def get_accuracy(model, loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()
            
    return correct / total

In [ ]:
acc_train = get_accuracy(model, dataloader_train)
acc_test = get_accuracy(model, dataloader_test)

print(f"正解率（学習データ）：{acc_train:.3f}")
print(f"正解率（評価データ）：{acc_test:.3f}")

## 75. 損失と正解率のプロット

> 問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

In [ ]:
def get_loss_and_accuracy(model, criterion, loader):
    model.eval()
    loss = 0.0
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()
    
    return loss / len(loader), correct / total


In [ ]:
# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義（確率的勾配降下法）
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# 学習
num_epochs = 30
log_train = []
log_valid = []

for epoch in range(num_epochs):
    # 訓練モードにする
    model.train()
    for i, (inputs, labels) in enumerate(dataloader_train):
        # 勾配を0で初期化
        optimizer.zero_grad()
        
        # 順伝播 + 誤差逆伝播 + 重み更新
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 損失と正解率の算出
    loss_train, acc_train = get_loss_and_accuracy(model, criterion, dataloader_train)
    loss_valid, acc_valid = get_loss_and_accuracy(model, criterion, dataloader_valid)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])
    
    # ログを出力
    print(f"epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}")  

In [ ]:
# 視覚化
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].plot(np.array(log_train).T[0], label="train")
ax[0].plot(np.array(log_valid).T[0], label="valid")
ax[0].set_xlabel("epoch")
ax[0].set_ylabel("loss")
ax[0].legend()
ax[1].plot(np.array(log_train).T[1], label="train")
ax[1].plot(np.array(log_valid).T[1], label="valid")
ax[1].set_xlabel("epoch")
ax[1].set_ylabel("accuracy")
ax[1].legend()
plt.show()

## 76. チェックポイント

> 問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．

In [ ]:
# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# 学習
num_epochs = 10
log_train = []
log_valid = []

for epoch in range(num_epochs):
    # 訓練モードにする
    model.train()
    for i, (inputs, labels) in enumerate(dataloader_train):
        # 勾配を0で初期化
        optimizer.zero_grad()
        
        # 順伝播 + 誤差逆伝播 + 重み更新
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 損失と正解率の算出
    loss_train, acc_train = get_loss_and_accuracy(model, criterion, dataloader_train)
    loss_valid, acc_valid = get_loss_and_accuracy(model, criterion, dataloader_valid)
    log_train.append([loss_train, acc_train])
    log_valid.append([loss_valid, acc_valid])
    
    checkpoint_path = os.path.join(CHPDIR, f"checkpoint{epoch + 1}.pt")
    
    # チェックポイントの保存
    torch.save({"epoch": epoch, "model_state_dict": model.state_dict(), 
                F"optimizer_state_dict": optimizer.state_dict()}, checkpoint_path)
    
    # ログを出力
    print(f"epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f}, accuracy_valid: {acc_valid:.4f}")  

## 77. ミニバッチ化

> 問題76のコードを改変し，B事例ごとに損失・勾配を計算し，行列Wの値を更新せよ（ミニバッチ化）．Bの値を1,2,4,8,…と変化させながら，1エポックの学習に要する時間を比較せよ．

In [ ]:
def train_model(
        dataset_train, dataset_valid, batch_size, model, 
        criterion, optimizer, num_epochs):
    # dataloaderの作成
    dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
    
    # 学習
    log_train = []
    log_valid = []

    for epoch in range(num_epochs):
        # 開始時刻の記録
        s_time = time.time()
        
        # 訓練モードにする
        model.train()
        for i, (inputs, labels) in enumerate(dataloader_train):
            # 勾配を0で初期化
            optimizer.zero_grad()

            # 順伝播 + 誤差逆伝播 + 重み更新
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # 損失と正解率の算出
        loss_train, acc_train = get_loss_and_accuracy(model, criterion, dataloader_train)
        loss_valid, acc_valid = get_loss_and_accuracy(model, criterion, dataloader_valid)
        log_train.append([loss_train, acc_train])
        log_valid.append([loss_valid, acc_valid])

        checkpoint_path = os.path.join(CHPDIR, f"checkpoint{epoch + 1}.pt")

        # チェックポイントの保存
        torch.save({"epoch": epoch, "model_state_dict": model.state_dict(), 
                    F"optimizer_state_dict": optimizer.state_dict()}, checkpoint_path)

        # 終了時刻の記録
        e_time = time.time()
        
        # ログを出力
        print(f"epoch: {epoch + 1}, loss_train: {loss_train:.4f}, accuracy_train: {acc_train:.4f}, loss_valid: {loss_valid:.4f},\
                accuracy_valid: {acc_valid:.4f}, {(e_time - s_time):.4f}sec")  
    
    return {"train": log_train, "valid": log_valid}

In [ ]:
# datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)

# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# モデルの学習
for batch_size in [2 ** i for i in range(11)]:
    print(f"バッチサイズ: {batch_size}")
    log = train_model(dataset_train, dataset_valid, batch_size, model, criterion, optimizer, 1)